<a href="https://colab.research.google.com/github/valentinamgavirira/analitica-/blob/main/RETO2ANALITICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DESCRIPCIÓN DE CADA VARIABLE

**TechSales_Reps **contienen registros de 21 990
representantes de ventas de los grupos de productos de hardware y software de una empresa de alta tecnología. Para cada em-pleado, los datos incluyen información sociodemográﬁ ca y educativa, el salario, el desempeño en ventas y un indicador de personalidad. También se incluye en los datos la puntuación neta del pro-motor, que es un indicador de la satisfacción del cliente con cada representante de ventas.Nombre de la variable Descripción o valores posiblesRep Una identiﬁ cación única para cada representante de ventas
**Business**Uno de los dos grupos de productos: Hardware y Software
**Age** Edad del empleado
**Female** 1 – Mujer0 – Caso contrari
**Years** El número de años que el empleado ha estado en la empresa
**College**Si el empleado tiene o no un título universitario de cuatro años (Sí/No)
**Personality** Analyst (Analista): este tipo de personalidad ejempliﬁ ca la racionalidad. Los analistas tienden a ser de mente abierta y de voluntad fuerte. Lesgusta trabajar de forma independiente y suelen abordar las cosasdesde una perspectiva muy práctica.Diplomat (Diplomático): los diplomáticos se preocupan por laspersonas y tienden a tener mucha empatía hacia los demás.Ejempliﬁ can la cooperación y la diplomacia.Explorer (Explorador): los exploradores son muy prácticos y puedenpensar rápidamente. Tienden a ser muy buenos para tomar decisionesrápidas y racionales en situaciones difíciles.Sentinel (Centinela): los centinelas son cooperativos y prácticos. Lesgusta la estabilidad, el orden y la seguridad. Las personas con estetipo de personalidad tienden a ser trabajadoras y meticulosas.
**Certificates **El número de certiﬁ caciones profesionales relevantes que ha obtenido cada empleado.**Feedback** El puntaje de retroalimentación promedio que cada empleado recibede sus compañeros y supervisor en la evaluación anual de 360 grados.Las puntuaciones posibles van de 0 (la más baja) a 4 (la más alta).**Salary** Sueldo base anual de cada empleado.
**NPS** El puntaje neto del promotor (NPS) es un indicador clave de lasatisfacción y lealtad del cliente

In [ ]:
#CARGO LAS LIBRERIAS
import numpy as np
import pandas as pd
#para implementación y evaluación del modelo
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import confusion_matrix, classification_report
from pydotplus import graph_from_dot_data

In [ ]:
#CARGO LA BASE DE DATOS
file_path="/content/2. TechSalesReport_Int_L.xlsx"
xls=pd.read_excel(file_path)

In [ ]:
df_reps = pd.read_excel(file_path, sheet_name="TechSales_Reps")
df_pronostico = pd.read_excel(file_path, sheet_name="Pronóstico")

In [ ]:
# Preprocesamiento de datos
df_reps["College"] = df_reps["College"].map({"Yes": 1, "No": 0})
df_reps["Business"] = df_reps["Business"].map({"Hardware": 0, "Software": 1})

In [ ]:
# Eliminar filas con valores nulos en College
df_reps = df_reps.dropna(subset=["College"])

In [ ]:
# Selección de variables predictoras y objetivo
X = df_reps.drop(columns=["Sales_Rep", "Personality", "College"])
y = df_reps["College"]

In [ ]:
# Verificar que no haya problemas con los datos
if X.empty or y.empty:
    raise ValueError("Error: No hay suficientes datos para entrenar el modelo. Verifica el archivo de entrada.")

In [ ]:
# Entrenamiento del modelo de Árbol de Decisión
clf = DecisionTreeClassifier(criterion="gini", max_depth=4, random_state=42)
clf.fit(X, y)

DecisionTreeClassifier(max_depth=4, random_state=42)

In [ ]:
total_personas = df_reps.shape[0]
inscritos = df_reps[df_reps["College"] == 1].shape[0]
no_inscritos = df_reps[df_reps["College"] == 0].shape[0]

print("Total de personas:", total_personas)
print("Número de personas inscritas:", inscritos)
print("Número de personas no inscritas:", no_inscritos)

Total de personas: 21990
Número de personas inscritas: 17520
Número de personas no inscritas: 4470


In [ ]:
# Despliegue del árbol de decisión
vs = X.columns.tolist()
dot_data = export_graphviz(clf, feature_names=vs, filled=True, rounded=True, special_characters=True)
graph = graph_from_dot_data(dot_data)
graph.write_png("TechSales_DecisionTree.png")

True

In [ ]:
print("Número de nodos terminales:", clf.get_n_leaves())

Número de nodos terminales: 16


In [ ]:
def contar_nodos_puros(arbol):
    hojas = arbol.tree_.children_left == -1  # Los nodos hoja tienen -1 en children_left
    impuridades = arbol.tree_.impurity[hojas]
    nodos_puros = sum(impuridades == 0)  # Un nodo puro tiene impureza = 0
    return nodos_puros

nodos_puros = contar_nodos_puros(clf)
print(f"Número de nodos puros: {nodos_puros}")

Número de nodos puros: 0


In [ ]:
# Predicción y evaluación del modelo
y_pred = clf.predict(X)
cm = confusion_matrix(y, y_pred)
print("Matriz de Confusión:\n", cm)

VN, FP, FN, VP = cm.ravel()

Matriz de Confusión:
 [[  114  4356]
 [   69 17451]]


In [ ]:
# Cálculo de métricas
Exactitud = (VP + VN) / (VP + VN + FP + FN)
Sensibilidad = VP / (VP + FN)
Especificidad = VN / (VN + FP)
Precision = VP / (VP + FP)
TasaError = (FP + FN) / (VP + VN + FP + FN)
PredNeg = VN / (VN + FN)if (VN + FN)

# Mostrar métricas
print("Exactitud:", Exactitud)
print("Sensibilidad:", Sensibilidad)
print("Especificidad:", Especificidad)
print("Precisión:", Precision)
print("Tasa de Error:", TasaError)
print("Tasa de Predicción Negativa:", PredNeg)

NameError: name 'VP' is not defined

In [ ]:
# Aplicar modelo a la base de pronóstico
df_pronostico["Business"] = df_pronostico["Business"].map({"Hardware": 0, "Software": 1})
X_forecast = df_pronostico.drop(columns=["Sales_Rep", "Personality"])

In [ ]:
# Verificar si hay datos para predecir
if not X_forecast.empty:
    df_pronostico["College_Predicted"] = clf.predict(X_forecast)
    df_pronostico["College_Predicted"] = df_pronostico["College_Predicted"].map({1: "Yes", 0: "No"})
else:
    print("Advertencia: No hay datos válidos en la base de pronóstico.")

In [ ]:
# Guardar resultados en un archivo Excel
output_path = "TechSales_Pronostico.xlsx"
df_pronostico.to_excel(output_path, index=False)

# Mostrar ruta del archivo generado
print("Archivo guardado en:", output_path)


Archivo guardado en: TechSales_Pronostico.xlsx


In [ ]:
# Filtrar los datos según la regla dada
filtro = (df_reps["Salary"] <= 64100) & (df_reps["Salary"] > 47100) & \
         (df_reps["Age"] > 39.5) & (df_reps["Certificates"] > 1.5)

# Contar el número total de personas que cumplen la condición
total_personas = df_reps[filtro].shape[0]

# Contar cuántas de esas personas tienen título universitario
personas_con_titulo = df_reps[filtro]["College"].sum()

# Mostrar resultados
print(f"Total de personas que cumplen la condición: {total_personas}")
print(f"Personas con título universitario dentro del intervalo: {personas_con_titulo}")

Total de personas que cumplen la condición: 1551
Personas con título universitario dentro del intervalo: 986


In [ ]:
VP

17451

In [ ]:
# Contar cuántas predicciones dieron 'Yes' (1)
num_yes = (df_pronostico["College_Predicted"] == "Yes").sum()

# Mostrar resultado
print(f"Número de personas pronosticadas con título universitario ('Yes'): {num_yes}")

Número de personas pronosticadas con título universitario ('Yes'): 16


In [ ]:
VN

114

In [ ]:
# Filtrar datos según la condición dada
filtro = (df_reps["Salary"] <= 64100) & (df_reps["Salary"] > 47100) & \
         (df_reps["Age"] > 39.5) & (df_reps["Certificates"] <= 1.5) & (df_reps["College"] == 0)

# Contar cuántas personas cumplen la condición
num_no_college = df_reps[filtro].shape[0]

# Mostrar el resultado
print(f"Número de personas sin título universitario en el intervalo dado: {num_no_college}")

Número de personas sin título universitario en el intervalo dado: 356


In [ ]:
print(f"Número de datos utilizados para entrenar el modelo: {X.shape[0]}")

Número de datos utilizados para entrenar el modelo: 21990


analisis. para la implementacion del modelo de arbol de decision se tomaron un total de 4 variables, para la construccion del modelo se tuvieron un total de 21990, de las cuales el Número de personas inscritas fueron 17520 y el Número de personas no inscritas fue de 4470. luego de la ejecucion del modelo se logro una exactitud cercana al 80%, tambien una precisión cercana al 80% y una tasa de error cercana al 20%; logramos